In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

1. # Libraries

In [ ]:
import pandas as pd
from scipy.misc import imread
import matplotlib.pyplot as plt
import imageio
path = "../input/"

2. # Importation of datasets

In [ ]:
breeds = pd.read_csv(path + 'breed_labels.csv')
colors = pd.read_csv(path + 'color_labels.csv')
states = pd.read_csv(path + 'state_labels.csv')


#cat_cols = ['Type', 'Gender', 'MaturitySize', 'FurLength',  'Vaccinated', 
#            'Dewormed', 'Sterilized', 'Health', 'Breed1', 'Breed2', 'Color1',
#            'Color2', 'Color3', 'State', 'AdoptionSpeed']

#dtypes = {
#    col: 'category'
#    for col in cat_cols
#}

train = pd.read_csv(path + 'train/train.csv')#, dtype=dtypes)
test = pd.read_csv(path + 'test/test.csv')#, dtype=dtypes)


In [ ]:
train.dtypes

The goal of the competition is to make predictions that have the following format.

In [ ]:
sample_sub = pd.read_csv(path +'test/sample_submission.csv')
sample_sub.head()

3. # Data Description

In [ ]:
train.shape

In [ ]:
train.describe()

## Data field

Quantitative : Age, Quantity, Fee, VideoAmt, PhotoAmt  
Qualitative : Name, Description  
Categorial : Type, Gender, MaturitySize, FurLength,  Vaccinated, Dewormed, Sterilized, Health  
Categorial (by a joiner) : Breed123, Color123, State  
Data to predict : Adoption Speed (0,1,2,3,4)  

## Creation of a categorial translator

In [ ]:
#Utilisation : translator['name_field'][id_cat]
translator = {}
translator['Type'] = ['No','Dog','Cat']
translator['Gender'] = ['Male', 'Female', 'Mixed']
translator['MaturitySize'] = ['Not Specified', 'Small', 'Medium', 'Large', 'Extra Large']
translator['FurLength'] = ['Not Specified', 'Short', 'Medium', 'Long']
translator['Vaccinated'] = ['Not Sure', True, False]
translator['Dewormed'] = ['Not Sure', True, False]
translator['Sterilized'] = ['Not Sure', True, False]
translator['Health'] = ['Not Specified','Healthy', 'Minor Injury', 'Serious Injury']
translator['AdoptionSpeed'] = ['on the same day', 'between 1 and 7 days', 'between 8 and 30 days', 'between 31 and 90 days', 'after 100 days']

## Example, with the first raw of the train dataset.

In [ ]:
indiv1 = train.loc[1704]

In [ ]:
indiv1

### Joins

In [ ]:
#Its breed
for numbreed in range(1,3):
    idb = indiv1['Breed'+ str(numbreed)]
    if idb != 0:
        print(breeds[breeds.BreedID==int(idb)])

In [ ]:
#Its color
for numcolor in range(1,4):
    idc = indiv1['Color'+ str(numcolor)]
    if idc != 0:
        print(colors[colors.ColorID==int(idc)])

In [ ]:
#Its State
print(states[states.StateID==int(indiv1.State)])

In [ ]:
translator['MaturitySize'][int(indiv1.MaturitySize)]

The first pet in the list is a small male cat, named Nibble. Its a black and white Tabby, came from Selangor. It was not vaccinated, not dewormed and not sterialized. It was healthy. His profile had 1 photo and 0 video. It was adopted in the 1st month after being listed.

In [ ]:
#Photos
for numphot in range(1, int(indiv1.PhotoAmt)+1):
    img = imageio.imread(path + 'train_images/' + str(indiv1.PetID) + '-' + str(numphot) + '.jpg')
    plt.imshow(img)
    plt.show()

## Some Graphs

In [ ]:
plt.bar(range (0,5), train['AdoptionSpeed'].value_counts())
plt.title("Number of pets by AdoptionSpeed categorial")

In [ ]:
#plt.subplot(1,2,1)
#plt.bar(translator['Type'][1:3], train['Type'].value_counts())
#plt.title("Distribution of pets by type")
#plt.ylabel('AdoptionSpeed')
#plt.subplot(1,2,2)
#adopsp1 = train[train.Type == 1]
#adopsp2 = train[train.Type == 2]
#mean1 = adopsp1['AdoptionSpeed'].mean()
#mean2 = adopsp2['AdoptionSpeed'].mean()
#plt.bar(translator['Type'][1:3], [mean1, mean2])
#plt.title ('Mean AdoptionSpeed by Type')
#plt.xlabel('Type')


# Feature extraction

Something that is always a good idea is to merge the training set with the test set. This makes the feature extraction code much more terse.

In [ ]:
train['is_train'] = True
test['is_train'] = False
all=pd.concat((train, test), sort = True)
all=all.reset_index(drop=True)
all = all.astype(test.dtypes)
#train.drop(['Description', 'RescuerID'], axis=1, inplace =True)

In [ ]:
assert all['AdoptionSpeed'].isnull().sum() == len(test)

Let's add some feature

In [ ]:
#df['garage_and_paved_driveway'] = df['GarageQual'].notnull() & df['PavedDrive'].notnull()
test1 = all[all.Age >= 8 ] 
test2 = all[all.Age < 8 ] 
print(len(test1)) ; print(len(test2))

In [ ]:
all['Young_and_Healthy'] = (all.Age < 8) & (all.Health == 1)

We'll now handle the missing values. For the categorical variables we'll add a new value called `'missing'`. As for the numerical variables we will impute them with their mean.

In [ ]:
all.dtypes

In [ ]:
from sklearn import preprocessing

cat_cols = all.select_dtypes('category').columns
for col in cat_cols:
    if all[col].isnull().sum() > 0:
        all[col] = all[col].cat.add_categories('missing').fillna('missing')
    all[col] = preprocessing.LabelEncoder().fit_transform(all[col])

In [ ]:
all.dtypes

# Machine Learning

A good practice is to first split the data between the features into `X_fit`, `y_fit`, and `X_test`.

In [ ]:
import numpy as np

to_drop = ['PetID', 'is_train', 'Description', 'Name', 'RescuerID']

X_train = all.query('is_train == True').drop(columns=to_drop + ['AdoptionSpeed']) #garde les lignes pour lesquels is_train=true
y_train = all.query('is_train == True')['AdoptionSpeed'] #sépare y_train
X_test = all.query('is_train == False').drop(columns=to_drop + ['AdoptionSpeed']) #garde les lignes pour lesquels is_train=false

X_train.head()

We can write some tests to make sure our data is okay.

In [ ]:
#On fait quelques verifs
#assert all(X_train.columns == X_test.columns)
assert X_train.isnull().sum().sum() == 0
assert X_test.isnull().sum().sum() == 0
assert len(X_train) == len(y_train)

Let's implement a linear regression model.

In [ ]:
from sklearn import decomposition
from sklearn import linear_model
from sklearn import pipeline
from sklearn import preprocessing

#Pipeline = décrire le modèle, ne pas le coder
#Transformeur = prend des données, appliques des opérations dessus, les renvoient
#Estimateur
model = pipeline.Pipeline([
    ('one_hot', preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore')), #dummies
    ('rescale', preprocessing.StandardScaler()), #centrer réduire
    ('pca', decomposition.TruncatedSVD(n_components=30)), #faire une ACP, reduction de dimensions
    ('ridge', linear_model.Ridge()) #Estimateur. Regressions linéaire où on pénalise les poids, on les force à être petit
    #Modèle plus timide, prédictions tournent plus autour de la moyenne, s'adapte mieux, va moins se tromper
])

To evaluate our model we can use a cross-validation scheme.

In [ ]:
#Méthode de validation croisée
#On s'entraine sur une partie, on teste sur une autre petite partie, on recommence plusieures fois
#On calcule la moyenne des erreurs
from sklearn import metrics
from sklearn import model_selection


def NegRMSE(y_true, y_pred):
    return -metrics.mean_squared_error(y_true, y_pred) ** 0.5

scoring = metrics.make_scorer(NegRMSE) 
cv = model_selection.KFold(n_splits=5, random_state=42)

scores = model_selection.cross_val_score(
    estimator=model,
    X=X_train,
    y=y_train,
    scoring=scoring,
    cv=cv
)

print('Model RMSE: {:.5f} ± {:.5f}'.format(-scores.mean(), scores.std()))
#Prochain but, tester des nouveaux paramètres du modèle

We can perform a grid-search to find better parameters for our model.

In [ ]:
#Lance une validation croisée pour chaque combinaison de paramètres possibles
param_grid = {
    'ridge__alpha': [0.01, 0.1, 1],
    'pca__n_components': [10, 25, 50]
}

grid = model_selection.GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    cv=cv,
    return_train_score=True
)

grid = grid.fit(X_train, y_train)

Let's put the results in a `DataFrame` to make them easier to read.

In [ ]:
#On affiche tous les résultats sous forme de tableau
results = pd.concat(
    (
        pd.DataFrame.from_dict(grid.cv_results_['params']),
        pd.DataFrame({
            'mean_train_score': -grid.cv_results_['mean_train_score'],
            'std_train_score': grid.cv_results_['std_train_score'],
            'mean_test_score': -grid.cv_results_['mean_test_score'],
            'std_test_score': grid.cv_results_['std_test_score']
        })
    ),
    axis='columns'
)

results.sort_values('mean_test_score')
#La meilleure combinaison de variable est celle qui minimine mean_test_score

`grid` now possesses an attribute called `best_estimator_`. We can use it to make our make our final predictions. 

In [ ]:
grid.best_estimator_

# Submitting

In [ ]:
sub = sample_sub.copy()

# We predict the log of the price
sub['AdoptionSpeed'] = grid.best_estimator_.predict(X_test)
sub['AdoptionSpeed'] = sub['AdoptionSpeed'].apply(round)

# We save the submission; the name of the file has the best validation 
sub.to_csv('submission.csv', index=False)